In [1]:
import pandas as pd
import requests
import json
from pymongo import MongoClient
import requests
import time
from collections import OrderedDict
from datetime import datetime, timedelta
import ipynb
from ipynb.fs.full.Coinbase_API import get_daily_data
from ipynb.fs.full.Into_the_Block_API import get_daily_info
from ipynb.fs.full.News_API import get_news
from ipynb.fs.full.ITB_Stats import get_daily_stats
import ipynb.fs.full.config as cfg

In [2]:
#Initialize database
client = MongoClient()
db = client.crypto

#Insert Tables
prices = db.prices
block_size = db.blocksize
avg_block_size = db.avgblocksize
n_trans = db.ntrans
n_payments = db.npayments
total_n_trans = db.totalntrans
med_confirm = db.medconfirm
avg_confirm = db.avgconfirm
news = db.news
stats = db.stats



def update_coinbase(database, collection, symbol, start_date_text='2015-07-30'):    
    status=True
    #Check if db has entries and update from the last update or the given date
    database_size = len(list(collection.find({}))) 
    #The the databse is empty update with all historical data
    if database_size == 0:
        start_date = datetime.strptime(start_date_text, '%Y-%m-%d').date()
        end_date = start_date + timedelta(days=275)
        end_date_text = end_date.strftime('%Y-%m-%d')
        entry = get_daily_data(symbol, start_date_text, end_date_text)
        while status == True: #Keep updating the dates until you return nothing
            start_date = end_date + timedelta(days=1)
            end_date = start_date + timedelta(days=275)
            start_date_text = start_date.strftime('%Y-%m-%d')
            end_date_text = end_date.strftime('%Y-%m-%d')
            entries = get_daily_data(symbol, start_date_text, end_date_text)
            if len(entries) > 0:
                entry.extend(entries)
                time.sleep(5)
            else:
                status=False
        collection.insert_many(entry)
    else: #otherwise update the entire database with the most recent entries
        last_update = list(collection.find({}, {'date':1,'_id':0}).sort([('date', -1)]).limit(1))[0]['date'].date()
        last_update_text = last_update.strftime('%Y-%m-%d')
        end_date = datetime.today().date() + timedelta(days=1)
        end_date_text = end_date.strftime('%Y-%m-%d')
        try:
            entries = get_daily_data(symbol, last_update_text, end_date_text)
            if len(entries) > 0 and last_update != datetime.today().date():
                collection.insert_many(entries)
                print('Updated')
            else:
                print('No update for today')
        except ValueError:
            pass
def update_ITB(chart, database, collection, start_date_text='2015-07-30'):
    status=True
    database_size = len(list(collection.find({})))
    if database_size == 0:
        entry = get_daily_info(start_date_text, chart)
        while status == True:
            start_date = datetime.strptime(start_date_text, '%Y-%m-%d').date()
            start_date = start_date + timedelta(days=365)
            start_date_text = start_date.strftime('%Y-%m-%d')
            try:
                entries = get_daily_info(start_date_text, chart)
                if len(entries) > 0:
                    entry.extend(entries)
                    time.sleep(5)
                else:
                    status=False
            except UnboundLocalError:
                status=False
                continue
        collection.insert_many(entry)
    else:
        last_update = list(collection.find({}, {'date':1,'_id':0}).sort([('date', -1)]).limit(1))[0]['date'].date()
        last_update = last_update + timedelta(days=1)
        last_update_text = last_update.strftime('%Y-%m-%d')
        try:
            entries = get_daily_info(last_update_text, chart)
            if len(entries) > 0:
                collection.insert_many(entries)
                print('Updated')
            else:
                print('No update for today')
        except ValueError:
            pass
def update_news(database, collection):
    entry = get_news()
    for i in entry:
        if len(list(collection.find({'url':i['url']}))) == 0 and len(i) > 0:
            collection.insert_one(i)
def update_stats(database, collection):
    entry = get_daily_stats()
    if len(list(collection.find({'date':entry[0]['date']}))) == 0 and len(entry) > 0:
        collection.insert_one(entry[0])

In [3]:
#Diagnose the DB if necessary

#View collections
# print(db.list_collection_names())

#Drop collections
# db.drop_collection('prices')

#Drop an entry
query = prices.find({}).sort([('date', -1)]).limit(3)
print(list(query))
#print(query[3])
#prices.delete_one(query[4])

#Check the length
#len(list(stats.find({})))

[{'_id': ObjectId('618801fb2e2473bee47db52a'), 'unix': 1636243200, 'low': 61397.86, 'high': 62400.0, 'open': 61539.3, 'close': 62211.47, 'volume': 3386.97941339, 'date': datetime.datetime(2021, 11, 7, 0, 0), 'vol_fiat': 210708968.1667296}, {'_id': ObjectId('618801fb2e2473bee47db52b'), 'unix': 1636156800, 'low': 60125.0, 'high': 61599.25, 'open': 61006.14, 'close': 61539.31, 'volume': 6366.43859189, 'date': datetime.datetime(2021, 11, 6, 0, 0), 'vol_fiat': 391786238.10228217}, {'_id': ObjectId('6186b53129fb9502ff968bb1'), 'unix': 1636156800, 'low': 60125, 'high': 61599.25, 'open': 61006.14, 'close': 60697.46, 'volume': 4780.19549623, 'date': datetime.datetime(2021, 11, 6, 0, 0), 'vol_fiat': 290145724.92460054}]


In [4]:
symbol = 'BTC-USD'
update_coinbase(db, prices, symbol)

Updated


In [5]:
chart = 'blocks-size'
update_ITB(chart, db, block_size)

https://api.blockchain.info/charts/blocks-size?&timespan=365days&start=2021-10-25&format=json
Updated


In [6]:
chart = 'avg-block-size'
update_ITB(chart, db, avg_block_size)

https://api.blockchain.info/charts/avg-block-size?&timespan=365days&start=2021-10-25&format=json
Updated


In [7]:
chart = 'n-transactions-per-block'
update_ITB(chart, db, n_trans)

https://api.blockchain.info/charts/n-transactions-per-block?&timespan=365days&start=2021-10-25&format=json
Updated


In [8]:
#Not working right now
#chart = 'n-payments-per-block'
#update_ITB(chart, db, n_payments)

In [9]:
chart = 'n-transactions-total'
update_ITB(chart, db, total_n_trans)

https://api.blockchain.info/charts/n-transactions-total?&timespan=365days&start=2021-10-25&format=json
Updated


In [10]:
chart = 'median-confirmation-time'
update_ITB(chart, db, med_confirm)

https://api.blockchain.info/charts/median-confirmation-time?&timespan=365days&start=2021-10-25&format=json
Updated


In [11]:
chart = 'avg-confirmation-time'
update_ITB(chart, db, avg_confirm)

https://api.blockchain.info/charts/avg-confirmation-time?&timespan=365days&start=2021-10-25&format=json
Updated


In [12]:
update_news(db, news)

In [13]:
update_stats(db, stats)

https://api.blockchain.info/stats


In [4]:
number1 = list(prices.find({}).sort('date', -1))[1]['close']

In [5]:
number1

60697.46